In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
import os
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from scipy import stats
import pickle
from utils.post_processing_utils import remove_manipulation_days, remove_bad_recordings, remove_exps_after_manipulations
from data_preprocessing.session_traces_and_mean import get_all_experimental_records, remove_experiments

In [2]:
def get_first_and_last_peaks(mouse, records, site='tail'):
    experiments_to_process = records[(records['mouse_id'] == mouse) & (records['recording_site'] == site)]
    sorted_records = experiments_to_process.sort_values('date').reset_index()
    first_recording = sorted_records.iloc[0]
    last_recording = sorted_records.iloc[-1]
    first_session_peak = get_mean_contra_peak(first_recording)
    last_session_peak = get_mean_contra_peak(last_recording)
    return first_session_peak, last_session_peak

In [3]:
def get_first_and_10000th_peaks(mouse, records, site='tail'):
    experiments_to_process = records[(records['mouse_id'] == mouse) & (records['recording_site'] == site)]
    sorted_records = experiments_to_process.sort_values('date').reset_index(drop=True)
    first_recording = sorted_records.iloc[0]
    last_recording_date = get_session_with_10000th_trial(mouse, sorted_records)
    last_recording_ind = sorted_records[sorted_records['date'] == last_recording_date].index.values[0]
    last_recording = sorted_records.iloc[last_recording_ind]
    first_session_peak = get_mean_contra_peak(first_recording)
    last_session_peak = get_mean_contra_peak(last_recording)
    return(first_session_peak, last_session_peak)

In [4]:
def get_mean_contra_peak(session_record):
    mouse_id = session_record['mouse_id']
    date = session_record['date']
    print(mouse, date)
    saving_folder = 'W:\\photometry_2AC\\processed_data\\' + mouse_id + '\\'
    aligned_filename = mouse_id + '_' + date + '_' + 'aligned_traces.p'
    save_filename = saving_folder + aligned_filename
    session_data = pickle.load(open(save_filename, "rb"))
    _trial_peaks = session_data.choice_data.contra_data.trial_peaks
    trial_peaks = [p if not isinstance(p, np.ndarray) else np.nan for p in _trial_peaks]
    mean_peak = np.nanmean(trial_peaks)
    return mean_peak

In [5]:
from utils.reaction_time_utils import get_bpod_trial_nums_per_session

In [14]:
def get_session_with_10000th_trial(mouse, experiments):
    dates = experiments[experiments['mouse_id']==mouse]['date'].unique()
    session_starts = get_bpod_trial_nums_per_session(mouse, dates)
    if session_starts[-1] >= 10000:
        last_session_idx = np.where(np.asarray(session_starts) >=10000)[0][0]
        print(np.asarray(session_starts) >=10000)
    else:
        last_session_idx = -1
        print(session_starts[-1])
    last_session_date = dates[last_session_idx]
    return(last_session_date)

In [15]:
mice = ['SNL_photo28', 'SNL_photo30', 'SNL_photo31', 'SNL_photo32', 'SNL_photo33', 'SNL_photo34', 'SNL_photo35']
records = get_all_experimental_records()
post_state_change_dates =  ['20200911', '20200915', '20200917', '20200918', '20200921', '20200829', '20200830', '20200831', '20201008','20201009', '20201007']
records_without_last = remove_experiments(records, {'SNL_photo17': ['20200317'], 'SNL_photo21': ['20201008','20201009','20200911','20200805', '20200915', '20200917', '20200918', '20200921','20200829', '20200830', '20200831','20201007'], 'SNL_photo22': post_state_change_dates, 'SNL_photo26': post_state_change_dates})

first_peaks = []
last_peaks = []
for mouse in mice:
    all_experiments = remove_exps_after_manipulations(records_without_last, [mouse])
    all_experiments = remove_bad_recordings(all_experiments)
    first, last = get_first_and_10000th_peaks(mouse, all_experiments, site='Nacc')
    first_peaks.append(first)
    last_peaks.append(last)


removing SNL_photo28: ['20201211' '20201214' '20201216' '20201218' '20201219' '20201221'
 '20201222' '20210119' '20210122' '20210127' '20210129' '20210202']
Int64Index([52, 96, 208, 514, 554, 569, 595], dtype='int64')
9917
SNL_photo28 20201118
SNL_photo28 20201209
removing SNL_photo30: ['20201211' '20201214' '20201216' '20201218' '20201219' '20201221'
 '20201222' '20210119' '20210122' '20210126' '20210127' '20210129'
 '20210202' '20210203' '20210204']
Int64Index([52, 96, 208, 514, 554, 569, 595], dtype='int64')
[False False False False False False False False False False  True]
SNL_photo30 20201117
SNL_photo30 20201209
removing SNL_photo31: ['20201211' '20201214' '20201216' '20201218' '20201219' '20201221'
 '20201222' '20210119' '20210122' '20210126' '20210127' '20210128'
 '20210129' '20210202' '20210203' '20210204']
Int64Index([52, 96, 208, 514, 554, 569, 595], dtype='int64')
8837
SNL_photo31 20201118
SNL_photo31 20201209
removing SNL_photo32: ['20201211' '20201214' '20201216' '202012

In [16]:
mice = ['SNL_photo16', 'SNL_photo17', 'SNL_photo18', 'SNL_photo21', 'SNL_photo22', 'SNL_photo26', 'SNL_photo37', 'SNL_photo43', 'SNL_photo44']
records = get_all_experimental_records()
post_state_change_dates =  ['20200911', '20200915', '20200917', '20200918', '20200921', '20200829', '20200830', '20200831', '20201008','20201009', '20201007']
records_without_last = remove_experiments(records, {'SNL_photo17': ['20200317'], 'SNL_photo21': ['20201008','20201009','20200911','20200805', '20200915', '20200917', '20200918', '20200921','20200829', '20200830', '20200831','20201007'], 'SNL_photo22': post_state_change_dates, 'SNL_photo26': post_state_change_dates})

first_peaks = []
last_peaks = []
for mouse in mice:
    all_experiments = remove_exps_after_manipulations(records_without_last, [mouse])
    all_experiments = remove_bad_recordings(all_experiments)
    first, last = get_first_and_10000th_peaks(mouse, all_experiments, site='tail')
    first_peaks.append(first)
    last_peaks.append(last)


removing SNL_photo16: ['20200313' '20200315' '20200318' '20200320' '20200322']
Int64Index([52, 96, 208, 514, 554, 569, 595], dtype='int64')
[False False False False False False False False  True  True  True]
SNL_photo16 20200210
SNL_photo16 20200307
removing SNL_photo17: ['20200311' '20200313']
Int64Index([52, 96, 208, 514, 554, 569, 595], dtype='int64')
[False False False False False False False False False False False False
 False False  True]
SNL_photo17 20200204
SNL_photo17 20200310
removing SNL_photo18: ['20200318' '20200320' '20200322']
Int64Index([96, 208, 514, 554, 569, 595], dtype='int64')
8071
SNL_photo18 20200223
SNL_photo18 20200316
Int64Index([52, 96, 208, 514, 554, 569, 595], dtype='int64')
[False False False False False False False False False False False False
  True  True]
SNL_photo21 20200806
SNL_photo21 20200908
Int64Index([52, 96, 208, 514, 554, 569, 595], dtype='int64')
[False False False False False False False False False False  True  True
  True  True]
SNL_photo

In [31]:
mice = ['SNL_photo28', 'SNL_photo30', 'SNL_photo31', 'SNL_photo32', 'SNL_photo33', 'SNL_photo34', 'SNL_photo35']
records = get_all_experimental_records()
post_state_change_dates =   ['20201211', '20201214','20201216', '20201218', '20201219', '20201221', '20201222']

exps_to_remove = {}
for mouse in mice:
    exps_to_remove[mouse] = post_state_change_dates
records_without_last = remove_experiments(records, exps_to_remove)
first_peaks = []
last_peaks = []
for mouse in mice:
    first, last = get_first_and_last_peaks(mouse, records_without_last, site='Nacc')
    first_peaks.append(first)
    last_peaks.append(last)

SNL_photo28 20201118
SNL_photo28 20210202


FileNotFoundError: [Errno 2] No such file or directory: 'W:\\photometry_2AC\\processed_data\\SNL_photo28\\SNL_photo28_20210202_aligned_traces.p'

In [53]:
data = pd.DataFrame({'mouse': mice, 'first session peak mean': first_peaks, 'last session peak mean': last_peaks})

In [37]:
data

,mouse,first session peak mean,last session peak mean
0,SNL_photo28,0.502072,1.252958
1,SNL_photo30,0.620741,0.620741
2,SNL_photo31,0.672864,0.990048
3,SNL_photo32,0.034759,0.112460
4,SNL_photo33,1.162061,1.162061
5,SNL_photo34,1.563913,1.292225
6,SNL_photo35,1.183479,1.320365


In [54]:
data.set_index('mouse').T

mouse,SNL_photo16,SNL_photo17,SNL_photo18,SNL_photo21,SNL_photo22,SNL_photo26,SNL_photo37,SNL_photo43,SNL_photo44
first session peak mean,1.532078,1.881998,2.303278,1.511586,1.515791,1.767145,2.014808,1.313025,1.397817
last session peak mean,1.174784,0.391713,1.081465,1.014487,1.376459,1.376231,1.288042,0.916336,1.283892


In [55]:
def francescas_plot(ax, data, show_err_bar=False, mean_linewidth=4, mean_line_color='blue', legend=False):
    """Plot lines for all mice from early to late.
    """
    data.plot(ax=ax, color='gray', legend=False)
    data.mean(1).plot(ax=ax,linewidth=mean_linewidth,color=mean_line_color)

    if show_err_bar:
        yerr = data.std(axis=1)

        plt.errorbar(np.array([0,1]), data.mean(1), yerr, color=mean_line_color, linewidth=4)

    # drop the left and bottom spine for cool looking effect
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)


In [47]:
import matplotlib

In [50]:
font = {'size': 6}
matplotlib.rc('font', **font)

fig, ax = plt.subplots(figsize=[2,2])
francescas_plot(ax, data.set_index('mouse').T, mean_line_color='#7FB5B5', mean_linewidth=6, show_err_bar=False)
plt.xticks([0, 1], ['First session peak', 'Last session peak'], fontsize=6)
plt.ylabel('Z-scored fluorescence', fontsize=6)

#significance stars
y = data.set_index('mouse').T.to_numpy().max() + .2
h = .1
plt.plot([0, 0, 1, 1], [y, y+h, y+h, y],c='k',lw=1)
ax.text(.5, y+h, '*', ha='center', fontsize=8)
ax.set_ylim([0.3, 2.6])
plt.tight_layout()

In [56]:
first_data = data['first session peak mean']
last_data = data['last session peak mean']
stat, pval = stats.ttest_rel(first_data, last_data)
#pval *= .5 # multiply by .5 bc doing a one-sided t-test! we have a hypothesis that it's going a certain way 
pval

0.005619024164589482

In [56]:
import os

filepath=os.path.join('W:\\photometry_2AC\\plots', 'nacc_quantified_peak_change_first_last.pdf' )
fig.savefig(filepath)